In [20]:
import folium
import pandas as pd
import math

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [4]:
#  Task 1: Mark all launch sites on a map
data = {
    'Name': ['KSC LC 39A', 'VAFB SLC-4E', 'CCSFS SLC-40', 'KSC LC 48'],
    'Lat': [28.561941, 34.632093, 28.563197, 28.608333],
    'Long': [-80.577236, -120.610829, -80.577002, -80.605833]
}
launch_sites_df = pd.DataFrame(data)

# Create a Folium map centered around the average coordinates of the launch sites
# Or choose a specific initial center, e.g., for the US
center_lat = launch_sites_df['Lat'].mean()
center_long = launch_sites_df['Long'].mean()
spacex_map = folium.Map(location=[center_lat, center_long], zoom_start=5)

# Add markers for each launch site
for index, row in launch_sites_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Name'],
        tooltip=row['Name']
    ).add_to(spacex_map)

# Display the map (in a Jupyter environment, this would render the map)
spacex_map

In [7]:
import requests
import io

URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"
try:
    resp = requests.get(URL)  
    resp.raise_for_status()
    #Read the content directly into pandas from the raw text
    spacex_df = pd.read_csv(io.StringIO(resp.text))
    print('Data read into a pandas dataframe!')
    print(spacex_df.head()) 
except requests.exceptions.RequestException as e:
    print(f"Error fetching data:{e}") 
except Exception as e:
    print(f"An unexpected error occurred:{e}")

Data read into a pandas dataframe!
   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        Lat  \
0      

In [8]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [9]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [10]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [11]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

In [12]:
# Task 2: Mark the success/failed launches for each site on the map
# Create a MarkerCluster object to group closely located markers
marker_cluster = MarkerCluster()

# Add the MarkerCluster to the map
site_map.add_child(marker_cluster)

# Iterate through each row in the spacex_df to add markers
for index, record in spacex_df.iterrows():
    # Determine the color based on the 'class' value
    # 'class' = 1 for success, 'class' = 0 for failure
    marker_color = 'green' if record['class'] == 1 else 'red'

    # Create a CircleMarker for each launch
    # You can use folium.Marker with custom icons if preferred,
    # but CircleMarker is often good for distinguishing success/failure visually.
    marker = folium.CircleMarker(
        location=[record['Lat'], record['Long']],
        radius=5, # Adjust radius as needed
        color=marker_color,
        fill=True,
        fill_color=marker_color,
        fill_opacity=0.7,
        popup=f"Launch Site: {record['Launch Site']}<br>"
              f"Outcome: {'Success' if record['class'] == 1 else 'Failure'}"
    )
# Add the marker to the marker_cluster
    marker_cluster.add_child(marker)

# Display the map
site_map

In [13]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [14]:
# Let's first create a MarkerCluster object
marker_cluster = MarkerCluster()

In [15]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [24]:
# Function to calculate distance between two points using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers

    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

# Define some hypothetical "proximity" points (e.g., nearby cities or coastline points)
# You would replace these with actual relevant proximity coordinates for your lab.
proximities = {
    'Cape Canaveral City': {'Lat': 28.3922, 'Long': -80.6077},
    'Houston (for KSC)': {'Lat': 29.7604, 'Long': -95.3698}, # Just an example if connecting KSC to Houston
    'Vandenberg Village (VAFB)': {'Lat': 34.7230, 'Long': -120.5758},
    'Titusville (KSC)': {'Lat': 28.6125, 'Long': -80.8089}
}

In [25]:
# Iterate through each launch site to calculate and visualize distances to a selected proximity
# For simplicity, let's pick one proximity for demonstration (e.g., Cape Canaveral City)
# In a real lab, you might have a dataset of proximities and iterate through them for each site.

# Let's add markers for these proximity points first, if they are not launch sites themselves
for name, coords in proximities.items():
    folium.Marker(
        location=[coords['Lat'], coords['Long']],
        icon=folium.Icon(color='blue', icon='info-sign'), # Blue marker for proximities
        popup=f"{name}"
    ).add_to(site_map) # Add to the main map, not the cluster, for clear distinction

# Calculate and visualize distance for each unique launch site to a selected proximity
# For this example, let's connect each launch site to the 'Cape Canaveral City' proximity
cape_canaveral_city_lat = proximities['Cape Canaveral City']['Lat']
cape_canaveral_city_long = proximities['Cape Canaveral City']['Long']

In [26]:
for index, site in launch_sites_df.iterrows():
    launch_site_lat = site['Lat']
    launch_site_long = site['Long']
    launch_site_name = site['Launch Site']

    # Calculate distance to Cape Canaveral City
    distance_to_proximity = calculate_distance(
        launch_site_lat, launch_site_long,
        cape_canaveral_city_lat, cape_canaveral_city_long
    )

    # Draw a PolyLine connecting the launch site to the proximity
    points = [[launch_site_lat, launch_site_long], [cape_canaveral_city_lat, cape_canaveral_city_long]]
    folium.PolyLine(
        locations=points,
        color='purple',
        weight=2,
        opacity=0.7,
        tooltip=f"{launch_site_name} to Cape Canaveral City: {distance_to_proximity:.2f} km"
    ).add_to(site_map)

    # Add a Text label or a small circle marker at the midpoint of the line showing the distance
    mid_lat = (launch_site_lat + cape_canaveral_city_lat) / 2
    mid_long = (launch_site_long + cape_canaveral_city_long) / 2

In [28]:
# Using a small circle marker with a popup for distance
folium.CircleMarker(
        location=[mid_lat, mid_long],
        radius=3,
        color='darkblue',
        fill=True,
        fill_color='darkblue',
        fill_opacity=1,
        popup=f"{distance_to_proximity:.2f} km"
    ).add_to(site_map)

# Display the map with distances
site_map